In [1]:
import numpy as np
import scipy.misc
import os,glob,cv2,shutil,argparse,matplotlib
import tensorflow as tf
from PIL import Image

In [4]:
#filledOmrimg=cv2.cvtColor(cv2.imread("./images/omr_filled/omr_filled_gujrat_3.png"), cv2.COLOR_BGR2GRAY)
#unfilledOmrimg=cv2.cvtColor(cv2.imread("./images/omr_unfilled/omr_unfilled_gujrat_3.png"), cv2.COLOR_BGR2GRAY)

filledOmrimg=cv2.cvtColor(cv2.imread("./images/omr_filled/eksteps_gj_test1.png"), cv2.COLOR_BGR2GRAY)
unfilledOmrimg=cv2.cvtColor(cv2.imread("./images/omr_unfilled/eksteps_gj_test2.png"), cv2.COLOR_BGR2GRAY)

#filledOmrGauimg = cv2.adaptiveThreshold(cv2.bitwise_not(cv2.GaussianBlur(filledOmrimg,(3,3),cv2.BORDER_DEFAULT)),maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 51, C = 0)
#unfilledOmrGauimg = cv2.adaptiveThreshold(cv2.bitwise_not(cv2.GaussianBlur(unfilledOmrimg,(3,3),cv2.BORDER_DEFAULT)),maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 51, C = 0)

filledOmrGauimg = cv2.adaptiveThreshold(cv2.bitwise_not(cv2.Canny(cv2.GaussianBlur(filledOmrimg,(3,3),cv2.BORDER_DEFAULT),80,100)),maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 51, C = 0)
unfilledOmrGauimg = cv2.adaptiveThreshold(cv2.bitwise_not(cv2.Canny(cv2.GaussianBlur(unfilledOmrimg,(3,3),cv2.BORDER_DEFAULT),80,100)),maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C, thresholdType = cv2.THRESH_BINARY, blockSize = 51, C = 0)

# filledCannyImg = cv2.Canny(filledOmrGauimg,80, 100)
# unfilledCannyImg = cv2.Canny(unfilledOmrGauimg,80, 100)

filledCount = cv2.countNonZero(filledOmrGauimg)
unfilledCount = cv2.countNonZero(unfilledOmrGauimg)

In [110]:
## Old logic
filledPercentage = filledCount / filledOmrGauimg.shape[1] * filledOmrGauimg.shape[0]
unfilledPercentage = unfilledCount / unfilledOmrGauimg.shape[1] * unfilledOmrGauimg.shape[0]

print('filledPercentage (old logic):',filledPercentage)
print('unfilled count (old logic):',unfilledPercentage)

filledPercentage (old logic): 641.0
unfilled count (old logic): 608.0


In [111]:
## New logic
filledPercentage = filledCount / (filledOmrGauimg.shape[1] * filledOmrGauimg.shape[0]) * 100
unfilledPercentage = unfilledCount / (unfilledOmrGauimg.shape[1] * unfilledOmrGauimg.shape[0]) * 100

print('filledPercentage (new logic):',filledPercentage)
print('unfilledPercentage (new logic):',unfilledPercentage)

if filledPercentage > 75.0 :
    print('filled')
else:
    print('unfilled')
    
if unfilledPercentage > 75.0 :
    print('filled')
else:
    print('unfilled')    

filledPercentage (new logic): 81.76020408163265
unfilledPercentage (new logic): 77.55102040816327
filled
filled


In [22]:
import keras#from keras.models import load_model
from tensorflow.keras import models
import numpy as np
import scipy.misc
import os,glob,cv2,shutil,argparse,matplotlib
import tensorflow as tf
from PIL import Image

## New Approach to check OMR Filled or no
def check_black_pixel(img_path):
    for img in glob.glob(img_path):
        filledimg=cv2.imread(img)
        filledimg=cv2.cvtColor(filledimg,cv2.COLOR_BGR2GRAY)
        #print(filledimg)
        number_of_black_pix = np.sum(filledimg <= 110)
        print('Number of black pixels in {}          '.format(img.split("/")[-1]), number_of_black_pix)
        if number_of_black_pix > 100 :
            print('filled')
        else:
            print('unfilled')
        
filled_img_path="./images/omr_filled/*.png"
print("black pixel count in filled image----------------------->>>>>>>")
check_black_pixel(filled_img_path)

unfilled_img_path="./images/omr_unfilled/*.png"
print("black pixel count in unfilled image--------------------->>>>>>")
check_black_pixel(unfilled_img_path)


black pixel count in filled image----------------------->>>>>>>
Number of black pixels in omr_filled_up_4.png           209
filled
Number of black pixels in omr_filled_up_5.png           167
filled
Number of black pixels in omr_filled_gujrat_13.png           285
filled
Number of black pixels in omr_filled_up_7.png           203
filled
Number of black pixels in omr_filled_up_6.png           217
filled
Number of black pixels in omr_filled_gujrat_3.png           501
filled
Number of black pixels in omr_filled_2.png           196
filled
Number of black pixels in omr_filled_1.png           542
filled
Number of black pixels in omr_filled_up_12.png           149
filled
Number of black pixels in omr_filled_up_10.png           182
filled
Number of black pixels in omr_filled_up_11.png           204
filled
Number of black pixels in up_omr_filled_up_internal.png           119
filled
Number of black pixels in up_omr_filled_up_internal1.png           0
unfilled
Number of black pixels in up_omr_fille

In [5]:
import keras#from keras.models import load_model
from tensorflow.keras import models
import numpy as np
import uuid
import scipy.misc
import os,glob,cv2,shutil,argparse,matplotlib
import tensorflow as tf
from PIL import Image
import imutils
from imutils.perspective import four_point_transform

save_path='./'
## New Approach to check OMR Filled or no
def check_black_pixel(img_path):
    for img in glob.glob(img_path):
        image_ori = cv2.imread(img)
        # gray it
        gray = cv2.cvtColor(image_ori, cv2.COLOR_BGR2GRAY)
        # blur it
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        
        (T, threshInv) = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
        print("T::",T)
        #print(" T - {} threshInv - {} ".format(T,threshInv))
        (T, threshInv) = cv2.threshold(gray, T-50, 255,cv2.THRESH_BINARY_INV)
        print("T::",T)
        print("threshInv ::",threshInv)
        print("threshInv.shape ::",threshInv.shape)
        nzCount =  cv2.countNonZero(threshInv)
        print(img.split("/")[-1],"----->>>white pixel count:   {}".format(nzCount))
    

        cv2.imwrite(save_path+img.split("/")[-1], threshInv)
#filled_img_path="./images/omr_filled/*/*.png"        
filled_img_path="./images/omr_filled/omr_filled_up_7.png"
print("black pixel count in filled image----------------------->>>>>>>")
check_black_pixel(filled_img_path)

#unfilled_img_path="./images/omr_unfilled/*/*.png"
unfilled_img_path="./images/omr_unfilled/omr_unfilled_2.png"
print("black pixel count in unfilled image--------------------->>>>>>")
check_black_pixel(unfilled_img_path)

# unfilled_img_path="./images/omr_unfilled2/*/*.jpg"
# print("black pixel count in unfilled image--------------------->>>>>>")
# check_black_pixel(unfilled_img_path)

black pixel count in filled image----------------------->>>>>>>
T:: 130.0
T:: 80.0
threshInv:: [[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 255 255 255
  255 255   